In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
print(dir2, dir1)
if not dir1 in sys.path: 
    sys.path.append(dir1)
import torch
from segmentation_pipeline import *

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

G:\Github Repositories\Segmentation-Pipeline\notebooks G:\Github Repositories\Segmentation-Pipeline
CUDA is available. Using GPU.


In [44]:
# Save data splits to a json file

from research.dmri_hippo.configs.main_config import get_context

variables = dict(DATASET_PATH="X:/Datasets/Diffusion_MRI/", CHECKPOINTS_PATH="X:/Checkpoints/")
context = get_context(device, variables)
context.init_components()

In [45]:
context.dataset.cohorts

{'all': RequireAttributes(attributes=['mean_dwi', 'md', 'fa']),
 'cross_validation': RequireAttributes(attributes=['fold']),
 'training': ComposeFilters(filters=[RequireAttributes(attributes=['fold']), ForbidAttributes(attributes={'fold': 0})]),
 'cbbrain_validation': ComposeFilters(filters=[RequireAttributes(attributes=['fold']), RequireAttributes(attributes={'fold': 0})]),
 'cbbrain_test': RequireAttributes(attributes={'cbbrain_test': True}),
 'ab300_validation': RequireAttributes(attributes={'ab300_validation': True}),
 'ab300_validation_plot': ComposeFilters(filters=[RequireAttributes(attributes={'ab300_validation': True}), RandomSelectFilter(num_subjects=20, seed=0)]),
 'cbbrain': RequireAttributes(attributes={'protocol': 'cbbrain'}),
 'ab300': RequireAttributes(attributes={'protocol': 'ab300'}),
 'rescans': ForbidAttributes(attributes={'rescan_id': 'None'}),
 'fasd': RequireAttributes(attributes={'pathologies': 'FASD'}),
 'inter_rater': RequireAttributes(attributes=['whole_roi_al

In [43]:
# Run final inference for hippo segmentation models

from research.dmri_hippo.hippo_inference import main

common_args = dict(
    ensemble_path='X:\\Checkpoints\\dmri-hippo-seg-v3\\augmentation_experiment_03_best_checkpoints\\standard\\',
    dataset_path='X:\\Datasets\\Diffusion_MRI\\',
    output_filename='standard',
    out_folder='X:\\Predictions\\Diffusion_MRI',
    device='cuda',
    ensemble_flips=True,
)

'''
main(
    **common_args,
    run_name='standard-test',
    ensemble_folds=True,
    cohort=ForbidAttributes(['fold'])
)'''

main(
    **common_args,
    run_name='standard-validation',
    ensemble_folds=False,
    cohort='cbbrain_validation'
)

using device cuda
Loaded models.
Running inference for context dmri-hippo-ceilings-mists-1gcb7co4
running inference for subjects: ['cbbrain_009', 'cbbrain_015', 'cbbrain_017', 'cbbrain_026']
cbbrain_009
Filled 0 voxels from detected holes.
Removed 0 voxels from 0 components.
cbbrain_015
Filled 1 voxels from detected holes.
Removed 0 voxels from 0 components.
cbbrain_017
Filled 1 voxels from detected holes.
Removed 4 voxels from 2 components.
cbbrain_026
Filled 0 voxels from detected holes.
Removed 0 voxels from 0 components.

running inference for subjects: ['cbbrain_043', 'cbbrain_054', 'cbbrain_056', 'cbbrain_078']
cbbrain_043
Filled 4 voxels from detected holes.
Removed 0 voxels from 0 components.
cbbrain_054
Filled 1 voxels from detected holes.
Removed 0 voxels from 0 components.
cbbrain_056
Filled 0 voxels from detected holes.
Removed 0 voxels from 0 components.
cbbrain_078
Filled 0 voxels from detected holes.
Removed 0 voxels from 0 components.

running inference for subjects: ['

In [38]:
# Add new ab300 subjects (380-390)
import json

in_path = Path('X:\\Datasets\\Diffusion_MRI_extra\\AB300_380-390')
in_files = list(in_path.iterdir())
out_path = Path('X:\\Datasets\\Diffusion_MRI\\subjects')
subject_ids = [380, 382, 383, 384, 385, 387, 388, 389, 390]

for subject_id in subject_ids:
    subject_name = f'ab300_{subject_id:03}'
    subject_path = out_path / subject_name
    subject_path.mkdir(exist_ok=True, parents=True)
    subject_files = [
        p for p in in_path.iterdir()
        if str(subject_id) in p.name
    ]
    
    image_names = ('mean_dwi', 'fa', 'md')
    for image_name, image_file_path in zip(image_names, subject_files):
        image = tio.ScalarImage(image_file_path)
        image.save(subject_path / f'{image_name}.nii.gz')
    
    with open(subject_path / "attributes.json", "w") as f:
        json.dump({
            "age": "None",
            "gender": "None",
            "pathologies": ["None"],
            "protocol": "ab300",
            "rescan_id": "None",
            "subject_id": subject_id
        }, f, indent=4)
    

In [21]:
from pathlib import Path
import torchio as tio

in_path = Path("X:\\Datasets\Diffusion_MRI\\tle_subject_original")
out_path = Path('X:\\Datasets\\Diffusion_MRI\\subjects')
subject_folders = [f'tle_{i:03}' for i in (11, 12, 14, 22, 23, 26, 27, 28, 29, 31, 33, 35, 36, 37, 38)]
#subject_folders.append('cbbrain_401')

for folder in subject_folders:
    subject_path = in_path / folder
    process_images = [('mean_dwi', tio.ScalarImage), 
                      ('fa', tio.ScalarImage), 
                      ('md', tio.ScalarImage),
                      ('whole_roi', tio.LabelMap),
                      ('whole_roi_alt', tio.LabelMap),]
    
    for image_name, image_constructor in process_images:
        image_path = subject_path / f'{image_name}.nii.gz'
        img = image_constructor(image_path)
        img.load()
        print(img)
        print(img.affine)
        
        #img_nib = nib.load(image_path)
        #print(img_nib)
        data = img.data
        if image_constructor == tio.ScalarImage:
            data = data.float()
        data = torch.flip(data, [1, 2])
        img.set_data(data)
        img.affine[:, 0:2] *= -1
        
        print(img.affine)
        print(img)
        img.save(out_path / folder / f'{image_name}.nii.gz')


ScalarImage(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: RAS+; memory: 7.3 MiB; dtype: torch.DoubleTensor)
[[   1.            0.            0.         -110.        ]
 [   0.            1.            0.         -108.        ]
 [   0.            0.            0.99999994   -9.99999905]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.            0.         -110.        ]
 [  -0.           -1.            0.         -108.        ]
 [  -0.           -0.            0.99999994   -9.99999905]
 [  -0.           -0.            0.            1.        ]]
ScalarImage(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
ScalarImage(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: RAS+; memory: 7.3 MiB; dtype: torch.DoubleTensor)
[[   1.            0.            0.         -110.        ]
 [   0.            1.            0.         -108.        ]
 [   0.            

In [ ]:
model = ModularUNet(
      in_channels=2,
      out_channels=2,
      filters=[40, 40, 80, 80, 120, 120],
      depth=6,
      block_params={'residual': True},
      downsample_class=BlurConv3d,
      downsample_params={'kernel_size': 3, 'stride': 2, 'padding': 1},
      upsample_class=BlurConvTranspose3d,
      upsample_params={'kernel_size': 3, 'stride': 2, 'padding': 1, 'output_padding': 0}
)

sum(p.numel() for p in model.parameters())

In [ ]:
N = 10
C = 3
H = W = D = 5
pred = torch.randn(N, C, H, W, D)
pred = torch.softmax(pred, dim=1)

refine = torch.randn(N, C, C, H, W, D)
refine = refine.softmax(dim=1)

refined = (refine * pred[:, None]).sum(dim=1)

check = refined.sum(dim=1)
check.max()

In [39]:
from pathlib import Path
import torchio as tio
import pandas as pd

raw_subjects_path = Path("X:\\Datasets\\Diffusion_MRI_extra\\Controls\\")
out_subjects_path = Path("X:\\Datasets\\Diffusion_MRI\\tle_subject_original\\")

for subject_path in raw_subjects_path.iterdir():
    subject_name = subject_path.name
    print(subject_name)

    for file_path in subject_path.iterdir():

        renames = [
            ("_FA", "fa", tio.ScalarImage),
            ("_MD", "md", tio.ScalarImage),
            ("_mean_DWI", "mean_dwi", tio.ScalarImage),
            ("_hippo", "whole_roi_alt", tio.LabelMap),
            ("_Whole_ROI_KGS", "whole_roi", tio.LabelMap),
        ]
        
        for old_name, new_name, image_constructor in renames:
            if old_name.lower() in file_path.name.lower():
                image = image_constructor(file_path)
                print(img)
                
                if new_name == "whole_roi":
                    image = CustomRemapLabels(remapping={2: 1}, masking_method="Right")(image)
                    image = CustomRemapLabels(remapping={1: 2}, masking_method="Left")(image)
                out_file_path = out_subjects_path / subject_name.lower() / f"{new_name}.nii.gz"
                out_file_path.parent.mkdir(exist_ok=True)
                print(img)
                continue
                image.save(out_file_path)
                print(out_file_path)


tle_011
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: torch.FloatTensor)
LabelMap(shape: (1, 220, 216, 20); spacing: (1.00, 1.00, 1.00); orientation: LPS+; memory: 3.6 MiB; dtype: tor

In [ ]:
import json
from pathlib import Path
import pandas as pd

subjects_path = Path("X:\\Datasets\\Diffusion_MRI\\subjects\\")
tle_subject_paths = [path for path in subjects_path.iterdir() if "tle" in path.name]

demographics_path = Path("X:\\Datasets\\Diffusion_MRI_extra\\Diffusion_MRI_old\\TLE_Validation_Data\\demographics.xlsx")
demographics_df = pd.read_excel(demographics_path)
demographics_df = demographics_df.set_index("Subject")

for subject_path in tle_subject_paths:
    print(subject_name)
    subject_name = subject_path.name.upper()
    subject_demographics = demographics_df.loc[subject_name]
    with open(subject_path / "attributes.json", "w") as f:
        json.dump({
            "age": int(subject_demographics['Age']),
            "gender": subject_demographics['Sex'],
            "pathologies": ["None"],
            "protocol": "cbbrain",
            "rescan_id": "None",
            "subject_id": int(subject_name.split("_")[1])
        }, f, indent=4)

In [ ]:
from research.dmri_hippo.configs.main_config import get_context

variables = dict(DATASET_PATH="X:/Datasets/Diffusion_MRI/", CHECKPOINTS_PATH="X:/Checkpoints/")
context = get_context(device, variables)
context.init_components()

In [ ]:
from pathlib import Path
import json
import shutil

task_name = "Task502_DMRI_Hippocampus_Whole_Split"
task_id = 502

cv_root = Path(f"X:\\Predictions\\nnUNet_trained_models\\nnUNet\\ensembles\\{task_name}")
long_name = "ensemble_2d__nnUNetTrainerV2__nnUNetPlansv2.1--3d_fullres__nnUNetTrainerV2__nnUNetPlansv2.1"

cv_nifti_path = cv_root / long_name / 'ensembled_postprocessed'
test_nifti_path = Path(f"X:\\Predictions\\nnUNet_trained_models\\nnUNet\\inference\\{task_name}\\predictionsTs\\ensemble")

nifti_file_paths = [
    path for path in list(cv_nifti_path.iterdir()) + list(test_nifti_path.iterdir())
    if path.suffix == ".gz"
]

subject_names_path = Path(f"X:\\Datasets\\nnUNet_raw_data_base\\nnUNet_raw_data\\{task_name}\\original_subject_names.json")
with subject_names_path.open() as f:
    names = json.load(f)
    cv_names = {v: k for k, v in subject_names['cross_validation_subjects'].items()}
    test_names = {v: k for k, v in subject_names['test_subjects'].items()}
    original_name_lookup = {**cv_names, **test_names}

out_path = Path("X:\\Predictions\\Diffusion_MRI\\nnUNet\\")

for file_path in nifti_file_paths:
    name = file_path.stem.split(".")[0]
    original_name = original_name_lookup[name]
    
    out_dir = out_path / original_name
    out_dir.mkdir(exist_ok=True)
    
    shutil.copy(file_path, out_dir / f"whole_roi_pred_task{task_id}.nii.gz")


In [ ]:
import copy
import torchio as tio
from pathlib import Path
import shutil

out_path = Path("X:\\Datasets\\Diffusion_MRI_cropped\\")

crop_transform = tio.CropOrPad((96, 88, 24), padding_mode='minimum', mask_name='whole_roi_union')

dataset = context.dataset.get_cohort_dataset("inter_rater")

for i, subject in enumerate(dataset.subjects):
    subject = copy.deepcopy(subject)
    subject.load()
    subject = crop_transform(subject)
    
    subject_path = Path(subject.folder)
    out_subject_path = out_path / "subjects" / subject['name']
    out_subject_path.mkdir(exist_ok=True)

    for image_name, image in subject.get_images_dict(intensity_only=False).items():
        if "uniform" in image and image['uniform']:
            continue
        image.save(out_subject_path / f"{image_name}.nii.gz")
    shutil.copy(subject_path / "attributes.json", out_subject_path / "attributes.json")

    if i == 0:
        uniform_path = out_path / "atlas"
        uniform_path.mkdir(exist_ok=True)
        for image_name, image in subject.get_images_dict(intensity_only=False).items():
            if "uniform" not in image or not image['uniform']:
                continue
            image.save(uniform_path / f"{image_name}.nii.gz")
            
    if i % 10 == 0:
        print(f"Saved subject {subject.name} {i}/{len(context.dataset)}")


In [ ]:
shape = context.dataset.subjects[0].shape
all_hippo = torch.zeros(size=shape, dtype=torch.int64)
N = 0

for subject in context.dataset.subjects:
    if 'whole_roi' in subject:
        N += 1
        all_hippo += (subject['whole_roi'].data > 0).int()

In [ ]:
import torchio as tio
from pathlib import Path

whole_roi_union = tio.LabelMap(tensor=(all_hippo != 0))
whole_roi_mean = tio.LabelMap(tensor=(all_hippo / N) > 0.5)

whole_roi_union.save(Path(context.dataset.root) / "atlas" / "whole_roi_union.nii.gz")
whole_roi_mean.save(Path(context.dataset.root) / "atlas" / "whole_roi_mean.nii.gz")

In [ ]:
from segmentation_pipeline.data_processing.dataset_fingerprint import *

get_bounds(whole_roi_union['data'][0])

In [ ]:
subject_fingerprints, summary_fingerprint = get_dataset_fingerprint(
    context.dataset,
    transform=EnforceConsistentAffine(),
    save=True,
    image_names=['whole_roi', 'hbt_roi', 'mean_dwi', 'md', 'fa']
)

In [ ]:
import json
from pathlib import Path

with (Path(context.dataset.root) / "fingerprint" / "subject_fingerprints.json").open() as f:
    subject_fingerprints = json.load(f)
    
with (Path(context.dataset.root) / "fingerprint" / "fingerprint.json").open() as f:
    fingerprint = json.load(f)

In [ ]:
import numpy as np

print(fingerprint['label_crop']['whole_roi']['all'])

crop_mean = np.array(fingerprint['label_crop']['whole_roi']['all']['median'])
crop_std = np.array(fingerprint['label_crop']['whole_roi']['all']['std'])

print(crop_mean)
print(crop_std)

for subject_name, fp in subject_fingerprints.items():
    try:
        subject_crop = np.array(fp['label_crop']['whole_roi']['all'])
    except:
        continue
    diff = np.abs(crop_mean - subject_crop)
    if np.any(diff > 3 * crop_std):
        print(subject_name, subject_crop, diff / crop_std)

In [ ]:
from pathlib import Path
import torchio as tio
import shutil

dataset = context.dataset.get_cohort_dataset("cbbrain")
subject_paths = {subject['subject_id']: subject['folder'] for subject in dataset.subjects}

def get_subject_id(file_path):
    out = file_path.stem
    out = out.split(".")[0]
    out = out.split("_")[2]
    out = int(out)
    return out

"""
full_dwi_path = Path("X:\\Datasets\\Diffusion_MRI_extra\\Diffusion_MRI_old\\Cb_Brain_n153\\Full")
for full_dwi_path in full_dwi_path.iterdir():
    subject_id = get_subject_id(full_dwi_path)
    out_path = Path(subject_paths[subject_id])
    
    full_dwi = tio.ScalarImage(full_dwi_path, channels_last=True)
    full_dwi.load()
    print(f"Saving {full_dwi_path} to {out_path}")
    full_dwi.save(out_path / "full_dwi.nii.gz")
"""

bvec_folder_path = Path("X:\\Datasets\\Diffusion_MRI_extra\\Diffusion_MRI_old\\Cb_Brain_n153\\Gradient_Info")
for bvec_path in bvec_folder_path.iterdir():
    subject_id = get_subject_id(bvec_path)
    out_path = Path(subject_paths[subject_id])
    shutil.copyfile(bvec_path, out_path / "full_dwi_grad.b")
    

In [ ]:
from pathlib import Path
import shutil

dest_path = Path("X:\\Predictions\\Diffusion_MRI\\isbi_models")

for subject in context.dataset.subjects:
    subject_path = Path(subject['folder'])
    
    dest_subject_path = dest_path / subject['name']
    dest_subject_path.mkdir(exist_ok=True)
    
    for subject_file in subject_path.iterdir():
        if "pred" in subject_file.name:
            shutil.move(subject_file, dest_subject_path / subject_file.name)